# Assignment 1

**Due to**: 11/01/2022 (dd/mm/yyyy)

If you deliver it by 11/12/2021 your assignment will be graded by 11/01/2022.


**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Summary**: Part-of Speech (POS) tagging as Sequence Labelling using Recurrent Neural Architectures

# Execution



In [34]:
import nltk
import pandas as pd
import numpy as np

nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

## 1. Corpus Pre-processing
NOTA: MOLTE COSE NON HANNO SENSO STO SPERIMENTANDO FUNZIONI PER VEDERE DI CAPIRCI QUALCOSA.
### 1.1 Vocabulary

Qui creo i tre sets taggati, un vocabolario per ognuno ed una lista di frequenze per ogni tag. Rimuovo i tag "-NONE-".

Nella cella successiva creo tre vocabolari con i tag codificati da un numero, non so per quale motivo ma lo faceva fabio. Online pare che utilizzino la lista di frequenze in questo caso, sostituendo il tag con la sua frequenza. 

Nella cella ancora successiva creo tre dataframe perché così chiesto dalla traccia.

In [35]:
fileids = nltk.corpus.treebank.fileids()


# Train
train_tagged_words = nltk.corpus.treebank.tagged_words(fileids[:101])
train_tagged_words = [t for t in train_tagged_words if t[1] != '-NONE-']
vocabulary_1 = list(set(train_tagged_words))

print('Train set:')
print('\t',len(train_tagged_words))
print('\t',len(vocabulary_1))

train_freqs = nltk.FreqDist(tag for (word, tag) in train_tagged_words)
# print(train_freqs.most_common())
# print(train_freqs.tabulate())

# Validation
val_tagged_words = nltk.corpus.treebank.tagged_words(fileids[101:151])
val_tagged_words = [t for t in val_tagged_words if t[1] != '-NONE-']
vocabulary_2 = list(set(val_tagged_words))

print('Validation set:')
print('\t',len(val_tagged_words))
print('\t',len(vocabulary_2))

val_freqs = nltk.FreqDist(tag for (word, tag) in val_tagged_words)
# print(val_freqs.most_common())

# Test
test_tagged_words = nltk.corpus.treebank.tagged_words(fileids[151:])
test_tagged_words = [t for t in test_tagged_words if t[1] != '-NONE-']
vocabulary_3 = list(set(test_tagged_words))

print('Test set:')
print('\t',len(test_tagged_words))
print('\t',len(vocabulary_3))

test_freqs = nltk.FreqDist(tag for (word, tag) in test_tagged_words)
# print(test_freqs.most_common())

Train set:
	 48183
	 8941
Validation set:
	 30576
	 6306
Test set:
	 15325
	 3827


In [36]:
tags = sorted(list(set([x[1] for x in vocabulary_1])))
tags = zip(tags, list(range(len(tags))))
tags_dict = dict(tags)


vocabulary_1_idx = [(x[0], tags_dict[x[1]]) for x in vocabulary_1]
vocabulary_2_idx = [(x[0], tags_dict[x[1]]) for x in vocabulary_2]
vocabulary_3_idx = [(x[0], tags_dict[x[1]]) for x in vocabulary_3]

In [37]:
df_train = pd.DataFrame(train_tagged_words, columns =['word', 'tag'])
df_val = pd.DataFrame(val_tagged_words, columns =['word', 'tag'])
df_test = pd.DataFrame(test_tagged_words, columns =['word', 'tag'])

In [38]:
print(len(train_freqs))
print(len(train_tagged_words))
print(len(vocabulary_1))

for t in train_freqs.most_common():
  print(t)

45
48183
8941
('NN', 6394)
('NNP', 5287)
('IN', 5051)
('DT', 4178)
('JJ', 3055)
('NNS', 3044)
(',', 2606)
('.', 1981)
('VBD', 1562)
('RB', 1515)
('CD', 1476)
('VB', 1211)
('CC', 1166)
('VBZ', 1153)
('VBN', 1059)
('TO', 1051)
('PRP', 957)
('VBG', 781)
('VBP', 734)
('MD', 419)
('PRP$', 412)
('``', 409)
('POS', 405)
("''", 399)
('$', 355)
(':', 294)
('WDT', 207)
('JJR', 160)
('WP', 142)
('RP', 140)
('NNPS', 98)
('JJS', 94)
('WRB', 92)
('RBR', 88)
('-RRB-', 56)
('-LRB-', 53)
('EX', 49)
('RBS', 20)
('LS', 10)
('PDT', 9)
('WP$', 6)
('FW', 2)
('UH', 1)
('SYM', 1)
('#', 1)


###1.2 GloVe 

In [39]:
import gensim
import gensim.downloader as gloader

emb_model = gloader.load('glove-wiki-gigaword-100')

[=================================================-] 99.6% 127.5/128.1MB downloaded


### 1.3 Model